**Topic**: Problem set 2 IO. 
**Author**: Jordi Torres-Vallverdú
**Date**: 13/01/2026
**Description**: This codes estimates the exercise 2 of the second problem set in IO. 

In [1]:
using StatFiles, DataFrames

#Input path
data_path = "/Users/jorditorresvallverdu/Library/Mobile Documents/com~apple~CloudDocs/tse/year2/term1/io/part2/jiadata.dta" ##-->CHANGE JUST THIS PART MANUALLY!

#Output path
export_dir = "/Users/jorditorresvallverdu/Documents/GitHub/TSE-2025-2026/IO/" ##Change this manually too

main = DataFrame(load(data_path))


Row,county,population,spc,urban,midwest,dbenton,southern,kmart,walmart,smallstores,dkmart,dwmart,x1,x2,x3,count0,count1,count2,nfirms
,Int16?,Float32?,Float32?,Float32?,Int8?,Float32?,Int8?,Int8?,Int8?,Int8?,Float32?,Float32?,Int8?,Float32?,Float32?,Int8?,Int8?,Int8?,Int8?
1,1,3.72,8.62,0.55,0,6.23,1,1,1,5,0.03,0.03,0,0.03,0.14,0,0,1,2
2,3,3.29,8.24,0.29,0,6.39,1,0,1,6,0.02,0.02,1,0.06,0.14,0,1,0,1
3,4,2.93,7.86,0.19,0,6.15,1,0,1,1,0.09,0.09,1,0.05,0.08,0,1,0,1
4,5,3.8,7.94,0.09,0,6.14,1,0,1,6,0.24,0.26,1,0.04,0.02,0,1,0,1
5,6,2.43,7.71,0.35,0,6.34,1,0,0,3,0.06,0.06,0,0.05,0.21,1,0,0,0
6,7,3.08,8.33,0.25,0,6.29,1,0,1,6,0.02,0.02,1,0.02,0.12,0,1,0,1
7,9,3.6,8.21,0.5,0,6.32,1,0,1,5,0.11,0.11,1,0.07,0.19,0,1,0,1
8,10,3.07,8.18,0.0,0,6.24,1,1,0,6,0.14,0.14,0,0.16,0.24,0,1,0,1
9,11,3.59,8.35,0.12,0,6.2,1,0,1,7,0.03,0.05,1,0.07,0.14,0,1,0,1


**Writing the inequalities**

It is quite simple to formulate the problem. As discussed above, 1,1 and 0,0 are not problematic. For these two we have these moments. 

1. (1,1) $\rightarrow$ $\mathbf{P}(\epsilon_{1}\geq -\pi_{1}^{D},\epsilon_{2}\geq -\pi_{2}^{D}, \rho )= \hat{P_{1,1}} \rightarrow M_1= \hat{P_{1,1}} -\mathbf{P}(\epsilon_{1}\geq -\pi_{1}^{D},\epsilon_{2}\geq -\pi_{2}^{D}, \rho )=0 $
2. (0,0) $\rightarrow$ $\mathbf{P}(\epsilon_{1}\leq -\pi_{1}^{M},\epsilon_{2}\leq -\pi_{2}^{M}, \rho )= \hat{P_{0,0}} \rightarrow M_2= \hat{P_{0,0}}- \mathbf{P}(\epsilon_{1}\leq -\pi_{1}^{M},\epsilon_{2}\leq -\pi_{2}^{M}, \rho )=0$

Here hat indicates that those are probabilities estimated with the observed data. This gives us two 4 moment inequalities (**Can we treat it as two equalities and that is all?**). 

Now let's consider the problematic cases (1,0), (0,1). For these, we have already shown that $P_{0,1}$ or $P_{1,0}$ can not be point identified in the data -unless we make assumptions on who moves first etc (as done above) because there is a region of multiplicity where multiple equilibria can rationalize the data. 

However, these probabilities are not devoid of empirical content. Let's take (1,0). We know that the probability we estimate in the data $\hat{P_{0,1}}$ will be bounded between: 

$P_{0,1}^{l} \leq \hat{P_{0,1}} \leq P_{0,1}^{u}  $, where the lower bound is just the region where 0,1 is equilibrium not in multiplicity. The upper bound is the opposite: the region when we assume the selection function of the multiplicity area always selects this outcome. This gives us two inequalities: 

1. $P_{0,1}^{l} \leq \hat{P_{0,1}} \rightarrow M_3= \hat{P_{0,1}}- P_{0,1}^{l} \geq 0 $
2. $\hat{P_{0,1}} \leq P_{0,1}^{u} \rightarrow M_4= P_{0,1}^{u} - \hat{P_{0,1}}  \geq 0$

Same exercise we can do with $P_{1,0}$. This leads with 8 moments, 2 equalities (4 inequalities) and 4 inequalities. 

----
If we pile up the moments into a vector $M(y; \theta)$, then we need to find values of the parameters that satisfy the six (in)equalities. Parameters that minimize the following function. To do this, without getting into inference, we can use different objective functions used in the literature. 

$$
Q(\theta)= \sum_{m} M(1,1; \theta)_m + M(0,0; \theta)_m + (M(0,1; \theta)_m^{u})_{+} + (M(0,1; \theta)_m^{l})_{-}  + (M(1,0; \theta)_m^{u})_{+} + (M(1,0; \theta)_m^{l})_{-}
$$


---
Related issue is that these moment inequalities are conditional on X. That is:

1. $ P(0,1|X)^{u} \leq P(0,1|X) \leq P(0,1|X)^{b} $


We follow Ciliberto/Tamer and we discritize the support of X/Z. 


In [2]:
main

names(main)


19-element Vector{String}:
 "county"
 "population"
 "spc"
 "urban"
 "midwest"
 "dbenton"
 "southern"
 "kmart"
 "walmart"
 "smallstores"
 "dkmart"
 "dwmart"
 "x1"
 "x2"
 "x3"
 "count0"
 "count1"
 "count2"
 "nfirms"

In [ ]:
using Statistics, GaussianDistributions, Distributions

function binormal(a, b, rho)
    d = BivariateNormal(0.0, 0.0, 1.0, 1.0, rho)
    return cdf(d, a, b)
end


function compute_probs(piw, pik, piduow, piduok, rho)
    p11_s= 1- (binormal(-piduow, 100, rho)) - (binormal(-piduow, -piduok, rho)) - (binormal(100, -piduok, rho)) 
    p00_s= (binormal(-piw, -pik, rho))
    p10_u_s= (binormal(100, -piduok, rho))- (binormal(-piw, -piduok, rho))
    p10_l_s= 1- (binormal(-piduow, -pik, rho))- p00 - p11
    p01_u_s= (binormal(-piduow, 100, rho))- (binormal(-piduow, -pik, rho))
    p01_l_s= 1- (binormal(-piw, -piduok, rho))- p00 - p11

    return p11_s, p00_s, p10_u_s, p10_l_s, p01_u_s, p01_l_s
end 


function compute_moments(p11_s, p00_s, p10_u_s, p10_l_s, p01_u_s, p01_l_s, p11, p00, p01, p10)

    m1= p00 - p00_s
    m2= p11- p11_s
    m3= p01 - p01_l_s #we penalize negative deviations here
    m4= p01 - p01_h_s #we penalize positive deviations here 
    m5= p10 - p10_l_s #penalize negative deviations
    m6= p01 - p01_h_s #we penalize positive deviations here 

    m= hcat(m1, 
        m2, 
        m3, 
        m4, 
        m5,
        m6
    )
    return m1, m2, m3, m4, m5, m6, m
end 


function compute_objective(m1, m2, m3, m4, m5, m6)
    Q= m1 + m2 + minimum(0,m3) + maximum(0,m4) + minimum(0,m5) + maximum(0,m6) #revise this, but seems quite accurate so far. 
    return Q
end 

compute_objective (generic function with 1 method)

In [ ]:
#code objective probability (non-parametrically, or condition on X)

using CategoricalArrays, StatsBase

main.joint_entry =
    string.(main.kmart, ",", main.walmart)

freq_pre = combine(groupby(main, :joint_entry),
               nrow => :n)
freq= freq_pre.n
prob_freq= freq ./ sum(freq)

#doubt, probability conditional on characteristics of the market? then we need to discretize the variables in X and make everything conditional. I think so, no?

#yes, this has to be conditional on X for sure. 
function discretize(x)
    breaks = quantile(x, [0.0,  0.5,  1.0])
    xc = cut(x, breaks; extend = true)
    return levelcode.(xc) #return in levels
end


main= transform(
    main, 
    :dbenton => discretize => :discrete_dbenton,
    :urban => discretize => :discrete_urban, 
    :spc => discretize => :discrete_spc, 
    :population => discretize =>:discrete_population
)
#re-estimate objective probabilities, per market. Estimate the counts of the data. 


#I should condition on discrete_urban, discrete_spc, discrete_population. And in the rest of the variables. 

function prob_conditional_nonparam()
    


end 

ArgumentError: ArgumentError: It is only allowed to pass a vector as a column of a DataFrame. Instead use `df[!, col_ind] .= v` if you want to use broadcasting.

In [ ]:

#Code moment function

#input: data, P_hat,  guess of parameters
#output: Q(theta)

function moments_solve(prob_freq, theta)



    #this depends on a given theta proposed. 
    β_pop, β_spc, β_urban, βw_dbenton, βw_southern, βk_midwest, γ_comp = theta

    #Just define the linear prediction
    piw= @. β_pop*pop + β_spc*spc + β_urban*urban +  βw_dbenton* dbenton + βw_southern* southern + γ_comp

    pik = @. β_pop*pop + β_spc*spc + β_urban*urban + βk_midwest* midwest + γ_comp
    
    piduow= @. β_pop*pop + β_spc*spc + β_urban*urban +  βw_dbenton* dbenton + βw_southern* southern 
    
    piduok = @. β_pop*pop + β_spc*spc + β_urban*urban + βk_midwest* midwest 

    #estimate probabilities, based on parameter guess. 

    p11_s, p00_s, p10_u_s, p10_l_s, p01_u_s, p01_l_s = compute_probs(piw, pik, piduow, piduok, rho)

    
    #probabilities computed from the data
    p11= prob_freq[1]
    p01= prob_freq[2]
    p00= prob_freq[3]
    p10= prob_freq[4]

    #build moments 
    m1, m2, m3, m4, m5, m6, m= compute_moments(p11_s, p00_s, p10_u_s, p10_l_s, p01_u_s, p01_l_s, p11, p00, p01, p10)


    #Define Q(θ)


    Q= compute_objective(m1, m2, m3, m4, m5, m6)


    return Q

end 



moments_solve (generic function with 1 method)

In [ ]:
#define a grid and then run for the grid. 

#I should drop for any missing variable here. 
pop= main.population
spc= main.spc
urban= main.urban
dbenton= main.dbenton
southern= main.southern
midwest= main.midwest


#define theta; informed by the reduced form? or define simply a grid, wide enough. 
G=50 #this defines the number of points of the grid per parameter, using the results from the previous model (adding + subtracting two std to define the grid.)
#make a decision on this object, cause currently it can't work at all. 
β_pop_guess = range(1.4, 2; length = G)
β_spc_guess=range(1.1, 2; length = G)
β_urban_guess=range(1, 2; length = G)
βw_dbenton= range(-0.8, -1.2; length= G)
βw_southern=range(0.3,0.7; length= G)
βk_midwest= range(0.1, 0.5; length= G)
γ_comp= range(-0.5,-3; length= G)

#define a grid for theta. Around 2 sd from their values in ordered logit? I can not loop over all possibilities... it is 50^7... so I can only partially identify the competition and set the rest fixed? but this is wrong. 
#this is maybe too inefficient. 

InterruptException: InterruptException:

In [13]:
prob_freq

4-element Vector{Float64}:
 0.13946731234866827
 0.3360774818401937
 0.47360774818401935
 0.05084745762711865